# By administrative: Land area exposure calculation

In [1]:
import os
import sys
import geopandas as gpd
import pandas
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
data_folder = "/Users/lena/OneDrive - OnTheHub - The University of Oxford/University/Oxford/OX_2018_2019/stlucia/analysis/python/projects/nismod-int-hack/data/Hackathon/rv"

### Loading in data 

In [3]:
def load_hazard(data_folder, hazard_id):  
    hazard_path = os.path.join(
        data_folder, 'data', 'hazards', '{}.shp'.format(hazard_id))
    print("Reading:", hazard_path)  
    hazards = gpd.read_file(hazard_path)
    
    if hazards.crs != {'init':'epsg:2006'}:
        hazards = hazards.to_crs({'init':'epsg:2006'})
    return hazards

In [4]:
hazard_ids = ['1m_sea-level', '4m_storm-surge', 'flashflooding', 'landslide_susceptibility'] 
#hazards = load_hazard(data_folder, hazard_id='')

administrative_path = os.path.join(
        data_folder, 'data','admin',  'stluc_administrative.shp')
administrative = gpd.read_file(administrative_path)

In [6]:
administrative

,id,admin_name,geometry
0,0,Anse-la-Raye,"POLYGON ((504489.1002518567 1543070.570047858,..."
1,1,Canaries,"POLYGON ((504962.9645831303 1534981.799464189,..."
2,2,Castries,"POLYGON ((504489.1002518567 1543070.570047858,..."
3,3,Choiseul,"POLYGON ((507850.020189992 1529299.173142115, ..."
4,4,Dennery,"POLYGON ((521289.1197935755 1543615.433000766,..."
5,5,Gros Islet,"POLYGON ((514096.4336815997 1543239.529323319,..."
6,6,Laborie,"POLYGON ((510296.0161504431 1518209.724388601,..."
7,7,Micoud,"(POLYGON ((519386.4274498464 1533181.09109453,..."
8,8,Soufriere,"POLYGON ((508887.6301650093 1530098.062839435,..."
9,9,Vieux Fort,"(POLYGON ((507850.020189992 1529299.173142115,..."


test  = df_intersections[df_intersections.admin_name == 'Castries']
df_intersections[df_intersections.hazard_id == hazard_id]
df_intersections.to_file(driver='ESRI Shapefile', filename='C:\\Users\\Lena\\Desktop\\delet\\DELETETHIS.shp')

In [5]:
def calculate_areas(administrative):
    administrative['area'] = administrative.area/1000000
    administrative = administrative [[
        'admin_name',
        'area', 
        'geometry'
    ]]
    return administrative

### Intersect admininstrative areas with each of the hazards

#### Keep appending on the rows as there will be a different number of hazards for each 

In [168]:
def intersect_admin_hazard2(administrative, hazard_df, df_intersections, hazard_id):
        
    for i,rows in administrative.iterrows():
        admin_geometry = rows.geometry
        admin_name = rows.admin_name
        print(admin_name)
        
        spatial_index_hazards = hazard_df.sindex
        possible_hazards_index = list(spatial_index_hazards.intersection(admin_geometry.bounds))
        possible_hazards = hazard_df.iloc[possible_hazards_index]
        precise_hazards = possible_hazards[possible_hazards.intersects(admin_geometry)]

        for j,jrows in precise_hazards.iterrows():

            intersection_shape = admin_geometry.intersection(jrows.geometry.buffer(0))
            intersection_area = intersection_shape.area

            df_intersections = df_intersections.append({
                'id':jrows.id,
                'geometry':intersection_shape, 
                'admin_name':admin_name, 
                'area_exp':intersection_area,
                'hazard_id':hazard_id
                }, ignore_index = True)
    
    return df_intersections

In [169]:
#calculate administrative areas (km2)
administrative = calculate_areas(administrative)

#initialise intersection results
df_intersections = gpd.GeoDataFrame(columns = ['id', 'geometry', 'admin_name', 'area_exp', 'hazard_id'], crs=administrative.crs)

for hazard_id in hazard_ids:
    print('>>> STARTING ',hazard_id)
    hazard_df = load_hazard(data_folder, hazard_id)
    df_intersections = intersect_admin_hazard2(administrative, hazard_df, df_intersections, hazard_id)

>>> STARTING  1m_sea-level
Reading: C:\Users\Lena\Dropbox\Hackathon\rv\data\hazards\1m_sea-level.shp
Anse-la-Raye
Canaries
Castries
Choiseul
Dennery
Gros Islet
Laborie
Micoud
Soufriere
Vieux Fort
>>> STARTING  4m_storm-surge
Reading: C:\Users\Lena\Dropbox\Hackathon\rv\data\hazards\4m_storm-surge.shp
Anse-la-Raye
Canaries
Castries
Choiseul
Dennery
Gros Islet
Laborie
Micoud
Soufriere
Vieux Fort
>>> STARTING  flashflooding
Reading: C:\Users\Lena\Dropbox\Hackathon\rv\data\hazards\flashflooding.shp
Anse-la-Raye
Canaries
Castries
Choiseul
Dennery
Gros Islet
Laborie
Micoud
Soufriere
Vieux Fort
>>> STARTING  landslide_susceptibility
Reading: C:\Users\Lena\Dropbox\Hackathon\rv\data\hazards\landslide_susceptibility.shp
Anse-la-Raye
Canaries
Castries
Choiseul
Dennery
Gros Islet
Laborie
Micoud
Soufriere
Vieux Fort


In [170]:
df_intersections.head

<bound method NDFrame.head of           id                                           geometry    admin_name  \
0        624  POLYGON ((503449.1627425908 1541625.682255677,...  Anse-la-Raye   
1       3057  POLYGON ((503459.1206188685 1541630.830786224,...  Anse-la-Raye   
2       2039  POLYGON ((503653.0408198946 1541795.082403573,...  Anse-la-Raye   
3        176  POLYGON ((503658.00940092 1541800.210889521, 5...  Anse-la-Raye   
4       2583  POLYGON ((503667.9053293439 1541820.684576474,...  Anse-la-Raye   
5       1554  POLYGON ((503672.8739022721 1541825.813065919,...  Anse-la-Raye   
6        390  POLYGON ((503677.8424732042 1541830.941556475,...  Anse-la-Raye   
7       2831  POLYGON ((503712.6636606544 1541856.624308329,...  Anse-la-Raye   
8        655  POLYGON ((503722.621398259 1541861.772951303, ...  Anse-la-Raye   
9       2988  POLYGON ((503811.6215205688 1542061.361672503,...  Anse-la-Raye   
10       550  POLYGON ((503806.5910878893 1542071.558225973,...  Anse-la-Raye  

In [172]:
def grouping(hazard_id):
    df_intersections_haz = df_intersections[df_intersections.hazard_id == hazard_id]
    df_intersections_admin = df_intersections_haz.groupby(['admin_name']).sum().reset_index()

    return df_intersections_admin

### Create dataframe with overview of area exposed for each hazard; delete admin_name (otherwise duplicated) and instead sort administrative areas by index

In [181]:
intersections_admin = []
for hazard_id in hazard_ids:
    
    intersections_admin_haz = grouping(hazard_id)
    intersections_admin_haz[hazard_id] = (intersections_admin_haz['area_exp']/1000000)
    intersections_admin_haz = intersections_admin_haz.drop(['area_exp'], axis=1)
    intersections_admin.append(intersections_admin_haz)
    
intersections_admin = pandas.concat(intersections_admin, axis=1, sort=False) 

intersections_admin = intersections_admin.drop(columns='admin_name')     
intersections_admin = intersections_admin.reset_index()
intersections_admin

#intersections_admin_1 = pandas.merge(administrative,intersections_admin, on=index)

,index,1m_sea-level,4m_storm-surge,flashflooding,landslide_susceptibility
0,0,0.107510,0.349298,4.489963,18.626602
1,1,0.056703,0.129775,0.643381,11.295275
2,2,0.702160,1.844444,4.701949,21.309316
3,3,0.042802,0.088691,0.592241,11.137471
4,4,0.211203,0.718461,3.859634,16.234605
5,5,0.791846,2.074472,2.710996,11.114375
6,6,0.135143,0.350658,1.043444,9.030778
7,7,0.679846,1.613307,2.502892,18.972560
8,8,0.113014,0.317421,1.096447,32.314789
9,9,1.116776,3.080519,1.496616,8.214233


In [182]:
administrative = administrative.reset_index()
administrative


,level_0,index,admin_name,area,geometry
0,0,0,Anse-la-Raye,44.126708,"POLYGON ((504489.1002518567 1543070.570047858,..."
1,1,1,Canaries,22.375241,"POLYGON ((504962.9645831303 1534981.799464189,..."
2,2,2,Castries,94.757798,"POLYGON ((504489.1002518567 1543070.570047858,..."
3,3,3,Choiseul,27.365623,"POLYGON ((507850.020189992 1529299.173142115, ..."
4,4,4,Dennery,85.112767,"POLYGON ((521289.1197935755 1543615.433000766,..."
5,5,5,Gros Islet,100.403328,"POLYGON ((514096.4336815997 1543239.529323319,..."
6,6,6,Laborie,34.926025,"POLYGON ((510296.0161504431 1518209.724388601,..."
7,7,7,Micoud,104.784416,"(POLYGON ((519386.4274498464 1533181.09109453,..."
8,8,8,Soufriere,53.877737,"POLYGON ((508887.6301650093 1530098.062839435,..."
9,9,9,Vieux Fort,46.272823,"(POLYGON ((507850.020189992 1529299.173142115,..."


In [183]:
intersections_admin_haz_all = pandas.merge(administrative,intersections_admin, on='index')
#intersections_admin_haz_all = intersections_admin_haz_all.drop(columns=['index', 'level_0'])
intersections_admin_haz_all

,level_0,index,admin_name,area,geometry,1m_sea-level,4m_storm-surge,flashflooding,landslide_susceptibility
0,0,0,Anse-la-Raye,44.126708,"POLYGON ((504489.1002518567 1543070.570047858,...",0.107510,0.349298,4.489963,18.626602
1,1,1,Canaries,22.375241,"POLYGON ((504962.9645831303 1534981.799464189,...",0.056703,0.129775,0.643381,11.295275
2,2,2,Castries,94.757798,"POLYGON ((504489.1002518567 1543070.570047858,...",0.702160,1.844444,4.701949,21.309316
3,3,3,Choiseul,27.365623,"POLYGON ((507850.020189992 1529299.173142115, ...",0.042802,0.088691,0.592241,11.137471
4,4,4,Dennery,85.112767,"POLYGON ((521289.1197935755 1543615.433000766,...",0.211203,0.718461,3.859634,16.234605
5,5,5,Gros Islet,100.403328,"POLYGON ((514096.4336815997 1543239.529323319,...",0.791846,2.074472,2.710996,11.114375
6,6,6,Laborie,34.926025,"POLYGON ((510296.0161504431 1518209.724388601,...",0.135143,0.350658,1.043444,9.030778
7,7,7,Micoud,104.784416,"(POLYGON ((519386.4274498464 1533181.09109453,...",0.679846,1.613307,2.502892,18.972560
8,8,8,Soufriere,53.877737,"POLYGON ((508887.6301650093 1530098.062839435,...",0.113014,0.317421,1.096447,32.314789
9,9,9,Vieux Fort,46.272823,"(POLYGON ((507850.020189992 1529299.173142115,...",1.116776,3.080519,1.496616,8.214233


In [184]:
admin_area_exp_path = os.path.join(
    data_folder, 'results', 'admin_area_exp.csv')

In [185]:
for hazard_id in hazard_ids:
    admin_area_exp = intersections_admin_haz_all
    admin_area_exp[hazard_id] = round(((admin_area_exp[hazard_id]/admin_area_exp['area'])*100))
admin_area_exp.to_csv(admin_area_exp_path)
admin_area_exp

,level_0,index,admin_name,area,geometry,1m_sea-level,4m_storm-surge,flashflooding,landslide_susceptibility
0,0,0,Anse-la-Raye,44.126708,"POLYGON ((504489.1002518567 1543070.570047858,...",0.0,1.0,10.0,42.0
1,1,1,Canaries,22.375241,"POLYGON ((504962.9645831303 1534981.799464189,...",0.0,1.0,3.0,50.0
2,2,2,Castries,94.757798,"POLYGON ((504489.1002518567 1543070.570047858,...",1.0,2.0,5.0,22.0
3,3,3,Choiseul,27.365623,"POLYGON ((507850.020189992 1529299.173142115, ...",0.0,0.0,2.0,41.0
4,4,4,Dennery,85.112767,"POLYGON ((521289.1197935755 1543615.433000766,...",0.0,1.0,5.0,19.0
5,5,5,Gros Islet,100.403328,"POLYGON ((514096.4336815997 1543239.529323319,...",1.0,2.0,3.0,11.0
6,6,6,Laborie,34.926025,"POLYGON ((510296.0161504431 1518209.724388601,...",0.0,1.0,3.0,26.0
7,7,7,Micoud,104.784416,"(POLYGON ((519386.4274498464 1533181.09109453,...",1.0,2.0,2.0,18.0
8,8,8,Soufriere,53.877737,"POLYGON ((508887.6301650093 1530098.062839435,...",0.0,1.0,2.0,60.0
9,9,9,Vieux Fort,46.272823,"(POLYGON ((507850.020189992 1529299.173142115,...",2.0,7.0,3.0,18.0
